In [ ]:
%%sh

# Specify an algorithm name
algorithm_name=autots
echo "Algorithm: " ${algorithm_name}

# Retreive account number
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo "Image URI: " ${image}


# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names ${algorithm_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${algorithm_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Build the docker image locally with the image name and tag it
# Finally, push image to ECR with the full image name.

docker build -t ${algorithm_name} .

docker tag ${algorithm_name} ${image}

docker push ${image}